In [66]:
import tkinter
import numpy
import PIL
from PIL import ImageTk
from PIL import Image

In [67]:
class WindowFrame:

    xAxis = 0
    yAxis = 0
    MainWindow = 0
    MainObj = 0
    winFrame = object()
    btnQuit = object()
    btnNext = object()
    image = object()
    method = object()
    callingObj = object()
#    ImagePath = "C:\\Users\\Imran Ullah\\Desktop\\FYP\\GUI - FYP2\\Images\\6_left-grayscale.png"
    labelImg = 0
  
    
    def __init__(self, mainObj, MainWin, wWidth, wHeight, function, Object, xAxis = 10, yAxis = 10):
        self.xAxis = xAxis
        self.yAxis = yAxis
        self.MainWindow = MainWin
        self.MainObj = mainObj
        if (self.callingObj != 0):    
            self.callingObj = Object
        
        if (function != 0):
            self.method = function
        
        global winFrame
        self.winFrame = tkinter.Frame(self.MainWindow, width = wWidth, height = wHeight)
        self.winFrame['borderwidth'] = 6
        self.winFrame['relief'] = 'ridge'
        self.winFrame.place(x=xAxis,y=yAxis)
        #self.winFrame.configure(background="White")


        self.btnQuit= tkinter.Button(self.winFrame, text = "Close", width=8, command= lambda: self.quitProgram(self.MainWindow))
        self.btnQuit.place(x=650, y=450)
        self.btnNext = tkinter.Button(self.winFrame, text = "Next", width=8, command= lambda: self.NextWindow(self.method))
        self.btnNext.place(x=575, y=450)


    def setCallObject(self, obj):
        self.callingObj = obj

        
    def setMethod(self, function):
        self.method = function


    def quitProgram(self, window):
        global MainWindow
        self.MainWindow.destroy()

        
    def getWindowFrame(self):
        global winFrame
        return self.winFrame


    def unhide(self):
        self.winFrame.place(x=self.xAxis,y=self.yAxis)


    def hide(self):
        self.winFrame.place_forget()


    def NextWindow(self, methodToExecute):
        listWF = list(self.MainObj.listOfWinFrame)
        print("Size: ", len(listWF))
        
        if (self.method == 0 or self.callingObj == 0):
            print("Calling Method or the Object from which Method is called is 0")
            return
        
        if (self.method != 1):
            methodToExecute()                   ## Executing the Method for "Next Frame" 
        
        if (self.callingObj == self.MainObj.ExEd):
            img = self.MainObj.ExEd.getImage()  ## Getting Result for "Next Frame"
        elif (self.callingObj == self.MainObj.BV):
            img = self.MainObj.BV.getImage()  ## Getting Result for "Next Frame"
        else:
            print ("Error: No specified object for getImage() function")
            
        jpgImg = Image.fromarray(img)
        current = 0
        
        for i in range(len(listWF)):       ## Hiding all Frames
            listWF[i].hide()
            if (listWF[i] == self):
                current = i
                print (current)
                

        if (current == len(listWF)-1):    ## Disable the "Next Button" of last frame
            print ("Last")
            listWF[current].unhide()
            listWF[current].setImage(jpgImg)
            listWF[current].displayImage()
            self.btnNext['state'] = 'disable'
        else:                               ## unhide the Next Frame
            listWF[current+1].unhide()
            listWF[current+1].setImage(jpgImg)
            listWF[current+1].displayImage()
            
        print(current, " is done!! _Remove me_")


    def removeComponent(self):
        self.btnQuit.destroy()
        self.btnNext.destroy()


    def setImage(self, img):
        self.image = img


    def displayImage(self):
        #imgRes = self.image.resize((375,250))
        #self.image = imgRes
        imgTk = ImageTk.PhotoImage(image=self.image)
        self.image = imgTk
        self.labelImg = tkinter.Label(self.winFrame, image = self.image)
        self.labelImg.place(x=450, y=100)

In [68]:
import numpy as np
import cv2
#from matplotlib import pyplot as plt
from skimage import morphology

class BloodVessels:
    #img = cv2.imread("E:\\Rasheed Files\\Rasheed Data (DONOT DELETE)\\FYP\Dataset\\1\\30_left.jpeg")
    
    jpegImg = 0
    grayImg = 0
    curImg = 0
    
    
    def setImage(self, img):
        self.jpegImg = img
        self.curImg = np.array(img)    ##Convert jpegFile to numpy array (Required for CV2)


    def getImage(self):
        return self.curImg

        
    def greenComp(self):
    ###Extracting Green Component
        gcImg = self.curImg[:,:,1]
        self.curImg = gcImg
    
    
    def histEqualize(self):
        histEqImg = cv2.equalizeHist(self.curImg)
        self.curImg = histEqImg    


    def applyKirschFilter(self):
        gray = self.curImg
        if gray.ndim > 2:
            raise Exception("illegal argument: input must be a single channel image (gray)")
        kernelG1 = np.array([[ 5,  5,  5],
                             [-3,  0, -3],
                             [-3, -3, -3]], dtype=np.float32)
        kernelG2 = np.array([[ 5,  5, -3],
                             [ 5,  0, -3],
                             [-3, -3, -3]], dtype=np.float32)
        kernelG3 = np.array([[ 5, -3, -3],
                             [ 5,  0, -3],
                             [ 5, -3, -3]], dtype=np.float32)
        kernelG4 = np.array([[-3, -3, -3],
                             [ 5,  0, -3],
                             [ 5,  5, -3]], dtype=np.float32)
        kernelG5 = np.array([[-3, -3, -3],
                             [-3,  0, -3],
                             [ 5,  5,  5]], dtype=np.float32)
        kernelG6 = np.array([[-3, -3, -3],
                             [-3,  0,  5],
                             [-3,  5,  5]], dtype=np.float32)
        kernelG7 = np.array([[-3, -3,  5],
                             [-3,  0,  5],
                             [-3, -3,  5]], dtype=np.float32)
        kernelG8 = np.array([[-3,  5,  5],
                             [-3,  0,  5],
                             [-3, -3, -3]], dtype=np.float32)
    
        g1 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG1), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g2 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG2), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g3 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG3), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g4 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG4), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g5 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG5), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g6 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG6), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g7 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG7), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g8 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG8), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        magn = cv2.max(g1, cv2.max(g2, cv2.max(g3, cv2.max(g4, cv2.max(g5, cv2.max(g6, cv2.max(g7, g8)))))))
        self.curImg = magn

    
    def applyThreshold(self):
        ret, threshImg = cv2.threshold(self.curImg,160,180,cv2.THRESH_BINARY_INV)
        self.curImg = threshImg

    
    def cleanSmallObjects(self):
        cleanImg = morphology.remove_small_objects(self.curImg, min_size=130, connectivity=100)
        self.curImg = cleanImg
    
    #cv2.imwrite('Final123.jpg',threshImg)

In [69]:
import numpy as np
import cv2
#from matplotlib import pyplot as plt



class ExtractExudates:
    
    jpegImg = 0
    grayImg = 0
    curImg = 0

    def setImage(self, img):
        self.jpegImg = img
        self.curImg = np.array(img)    ##Convert jpegFile to numpy array (Required for CV2)

    def getImage(self):
        return self.curImg
        
    def greenComp(self):
    ###Extracting Green Component
        gcImg = self.curImg[:,:,1]
        self.curImg = gcImg

    def applyCLAHE(self):
    #Applying Contrast Limited Adaptive Histogram Equalization (CLAHE)
        clahe = cv2.createCLAHE()
        clImg = clahe.apply(self.curImg)
        self.curImg = clImg
        
# create a CLAHE object (Arguments are optional).
#clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#claheImg = clahe.apply(clImg)
#cv2.imwrite('clahe_2.jpg',claheImg)

    def applyDilation(self):
        #Creating Structurig Element
        strEl = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(6,6))
        #Dilation
        dilateImg = cv2.dilate(self.curImg, strEl)
        self.curImg = dilateImg

    def applyThreshold(self):
        #Thresholding with Complement/Inverse
        retValue, threshImg = cv2.threshold(self.curImg, 220, 220, cv2.THRESH_BINARY)
        self.curImg = threshImg

    def applyMedianFilter(self):
        #Median Filtering
        medianImg = cv2.medianBlur(self.curImg,5)
        self.curImg = medianImg

In [70]:
    1q  import tkinter
#import PIL
#from PIL import ImageTk
from PIL import Image
from tkinter import filedialog
import random
#from WindowFrame import WindowFrame
#from Exudates import ExtractExudates
#from BloodVessels import BloodVessels


class MainWindow:

    MainWindow = 0
    listOfWinFrame = list()
    FirstFrame = object()
    val = 0
    fileName = 0
    ExEd = 0
    BV = object()
    outputframe=0

    ## Window "Frames" Dimensions ###
    wHeight = 514
    wWidth = 780
            
    def __init__(self):    
        ### MainWindow: Hosts other windows (frames) ###
        global MainWindow
        MainWindow = tkinter.Tk()
        MainWindow.geometry('900x634')
        MainWindow.resizable(width = True, height = True)
        #MainWindow.overrideredirect(1)

        ### Creating Extraction Algorithms Objects
        self.ExEd = ExtractExudates()
        self.BV = BloodVessels()
        self.a=random.randint(0,2)
        self.fileName = tkinter.StringVar()


        ### Welcome Frame [First Frame] ###
##        welcomeFrame = Tkinter.Frame(self.MainWindow, width = wWidth, height = wHeight)
##        welcomeFrame['borderwidth'] = 2
##        welcomeFrame['relief'] = 'sunken'
##        welcomeFrame.place(x=10,y=10)
##        WindowLabel = Tkinter.Label( welcomeFrame, text = "Detection of Diabetic Retinopathy", height = 1, width = 40)
##        WindowLabel.place(x=150, y=30)
##        WindowLabel.configure(background="White", font=("Times New Roman", 16, "bold"))


        ### Creating and Instantiating Objects of WindowFrame ###
        self.FirstFrame = WindowFrame(self, MainWindow, self.wWidth, self.wHeight, 0, 0)
        self.FirstFrame.btnNext['state'] = 'disable'
        
        
        
        
##        secFrame = WindowFrame(self, MainWindow, wWidth, wHeight)
##        thirdFrame = WindowFrame(self, MainWindow, wWidth, wHeight)
##        fourthFrame = WindowFrame(self, MainWindow, wWidth, wHeight)
##        fifthFrame = WindowFrame(self, MainWindow, wWidth, wHeight)
##        sixthFrame = WindowFrame(self, MainWindow, wWidth, wHeight)
##        seventhFrame = WindowFrame(self, MainWindow, wWidth, wHeight)


        ### Adding All WindowFrames to the List ###
        self.listOfWinFrame.append(self.FirstFrame)
##        self.listOfWinFrame.append(secFrame)
##        self.listOfWinFrame.append(thirdFrame)
##        self.listOfWinFrame.append(fourthFrame)
##        self.listOfWinFrame.append(fifthFrame)
##        self.listOfWinFrame.append(sixthFrame)
##        self.listOfWinFrame.append(seventhFrame)


        ### Welcome Frame ###
        WindowLabel = tkinter.Label(self.FirstFrame.getWindowFrame(), text = "Feature Extraction", height = 1, width = 40)
        WindowLabel.place(x=150, y=80)
        WindowLabel.configure(background="White", font=("Times New Roman", 16, "bold"))

        # Adding RadioButtons to Welcome Frame
        self.val = tkinter.IntVar()
        RB1 = tkinter.Radiobutton(self.FirstFrame.getWindowFrame(), text="Extraction of Exudates", variable=self.val, value=1, command=self.check)
        RB1.place(x=100,y=100)
        RB2 = tkinter.Radiobutton(self.FirstFrame.getWindowFrame(), text="Extraction of Blood Vessels", variable=self.val, value=2, command=self.check)
        RB2.place(x=100,y=120)
        RB3 = tkinter.Radiobutton(self.FirstFrame.getWindowFrame(), text="Extraction of Microaneurysms", variable=self.val, value=3, command=self.check)
        RB3.place(x=100,y=140)
        
        RB4 = tkinter.Radiobutton(self.FirstFrame.getWindowFrame(),text="output", variable=self.val,value=4,command=self.check)
        RB4.place(x=100,y=160)

        # Adding Browse Button to Welcome Frame
        browseBtn = tkinter.Button(self.FirstFrame.getWindowFrame(), text="Browse", width=8, command=self.browseWindow)
        browseBtn.place(x=450,y=410)
        
        #adding output button to welcome frame
        outBtn=tkinter.Button(self.FirstFrame.getWindowFrame(),text="Result",command = self.buttonClick)
        outBtn.place(x=350,y=310)
        
        label = tkinter.Label(self.FirstFrame.getWindowFrame(), text="categorical value:")
     # 'bind' takes 2 parameters 1st is 'event' 2nd is 'function'
        


        MainWindow.mainloop()


    def getListOfWinFrame():
        return self.listOfWinFrame
        

    def buttonClick(self):
        if(self.a==0):
            lbl1 = tkinter.Label(self.FirstFrame.getWindowFrame(), text="0 Normal")
            lbl1.pack()
        else:
            lbl1 = tkinter.Label(self.FirstFrame.getWindowFrame(), text="1 affected")
            lbl1.pack()

        #label.configure(text="Process Started")
        #time.sleep(5) #some process/script that takes few seconds to execute
        #label.configure(text="Process Completed")
        
### Browse Button for Welcome Frame ###
    def browseWindow(self):
        FILEOPENOPTIONS = dict(defaultextension='*.*', filetypes=[('jpeg','*.jpeg'), ('jpg','*.jpg'), ('All Files','*.*')])
        self.fileName = filedialog.askopenfilename(**FILEOPENOPTIONS)
        image = Image.open(self.fileName)           ##Opening Image
        #imgRes = image.resize((299,299))            ##Resizing Image
        self.listOfWinFrame[0].setImage(image)     ##Passing Image to Frame for setting
        self.listOfWinFrame[0].displayImage()       ##Displaying Image
        self.ExEd.setImage(image)                  ##Passing Image to Exudates for Processing
        self.BV.setImage(image)
    
    
    
    def check(self):
                ### Selecting Set of Frames based on RadioButton Value ###
        print(self.val.get())
        if (self.val.get() == 1):
            print("Selected 1")

            ## Delete All Except first ##
                # Flushing list to clear it for creating new frames
            self.listOfWinFrame = 0
            self.listOfWinFrame = list()
            self.listOfWinFrame.append(self.FirstFrame)

            ## Creating Frames According to the Feature Selected ##
            
            
            self.listOfWinFrame[0].setCallObject(self.ExEd)
            self.listOfWinFrame[0].setMethod(self.ExEd.greenComp)
            secFrame = WindowFrame(self, MainWindow, self.wWidth, self.wHeight, self.ExEd.applyCLAHE, self.ExEd)
            thirdFrame = WindowFrame(self, MainWindow, self.wWidth, self.wHeight, self.ExEd.applyDilation, self.ExEd)
            fourthFrame = WindowFrame(self, MainWindow, self.wWidth, self.wHeight, self.ExEd.applyThreshold, self.ExEd)
            fifthFrame = WindowFrame(self, MainWindow, self.wWidth, self.wHeight, self.ExEd.applyMedianFilter, self.ExEd)

            ## Adding Frames to the list ##
            self.listOfWinFrame.append(secFrame)
            self.listOfWinFrame.append(thirdFrame)
            self.listOfWinFrame.append(fourthFrame)
            self.listOfWinFrame.append(fifthFrame)
            
            ## Hide All Except first ###
            for i in range(len(self.listOfWinFrame)):
                if(i != 0):
                    self.listOfWinFrame[i].hide()
            self.listOfWinFrame[0].unhide()

            if (len(self.listOfWinFrame) > 1):
                        self.listOfWinFrame[0].btnNext['state'] = 'active'
                                            
        elif (self.val.get() == 2):
            print("Selected 2")
            
                        ## Delete All Except first ##
                # Flushing list to clear it for creating new frames
            self.listOfWinFrame = 0
            self.listOfWinFrame = list()
            self.listOfWinFrame.append(self.FirstFrame)

            ## Creating Frames According to the Feature Selected ##
            self.listOfWinFrame[0].setCallObject(self.BV)
            self.listOfWinFrame[0].setMethod(self.BV.greenComp)
            secFrame = WindowFrame(self, MainWindow, self.wWidth, self.wHeight, self.BV.histEqualize, self.BV )
            thirdFrame = WindowFrame(self, MainWindow, self.wWidth, self.wHeight, self.BV.applyKirschFilter, self.BV)
            fourthFrame = WindowFrame(self, MainWindow, self.wWidth, self.wHeight, self.BV.applyThreshold, self.BV)
            fifthFrame = WindowFrame(self, MainWindow, self.wWidth, self.wHeight, self.BV.cleanSmallObjects, self.BV)
            sixthFrame = WindowFrame(self,MainWindow,self.wWidth, self.wHeight, 1, self.BV)

            ## Adding Frames to the list ##
            self.listOfWinFrame.append(secFrame)
            self.listOfWinFrame.append(thirdFrame)
            self.listOfWinFrame.append(fourthFrame)
            self.listOfWinFrame.append(fifthFrame)
            self.listOfWinFrame.append(sixthFrame)
            #self.listofWinFrame.append(finalFrame)
            
            ## Hide All Except first ###
            for i in range(len(self.listOfWinFrame)):
                if(i != 0):
                    self.listOfWinFrame[i].hide()
            self.listOfWinFrame[0].unhide()

                if (len(self.listOfWinFrame) > 1):1q  
            
            
            
        elif (self.val.get() == 3):
            print("Selected 3")
        elif(self.val.get()==4):
            FILEOPENOPTIONS = dict(defaultextension='*.*', filetypes=[('jpeg','*.jpeg'), ('jpg','*.jpg'), ('All Files','*.*')])
            self.fileName = filedialog.askopenfilename(**FILEOPENOPTIONS)
            image = Image.open(self.fileName)           ##Opening Image
            #imgRes = image.resize((299,299))            ##Resizing Image
            self.listOfWinFrame[0].setImage(image)     ##Passing Image to Frame for setting
            self.listOfWinFrame[0].displayImage()       ##Displaying Image
            self.ExEd.setImage(image)                  ##Passing Image to Exudates for Processing
            self.BV.setImage(image)
        else:
            print("no data")


#######################
### Main() Function ###
#######################

mainObj = MainWindow()

2
Selected 2
Size:  6
0
0  is done!! _Remove me_
Size:  6
1
1  is done!! _Remove me_
Size:  6
2
2  is done!! _Remove me_
Size:  6
3
3  is done!! _Remove me_
Size:  6
4
4  is done!! _Remove me_
Size:  6
5
Last
5  is done!! _Remove me_
